<a href="https://colab.research.google.com/github/aniekanekanem/23-24-WSL-Stats-Analysis/blob/master/Optimizing_Marketing_Campaigns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are two categories of generating sales for a business which are organic sales and inorganic sales.  Organic sales occur naturally, they are not forced into growing the business.  However, when a business decides to intentionally grow its revenue base, it starts adopting various methods to force a sales growth.  This leads to inorganic sales; here we see the use of marketing methods such as cold emails, phone, flyers, social media, sales visit, etc.  The revenue generated here is an in-organic revenue.

Marketing Strategy Analysis

Programming Script and Technical Report

Table of Content
1. Introduction
2. Data loading and Quality
3. Feature Additions and Engineering
4. Exploratory Data Analysis and Statistical Analysis
5. Final Recommendations (Optimal Sales)

1. Introduction

	* What is the impact of each marketing strategy and sales visit on Sales (Amount Collected)?
	* Is the same strategy valid for all the different client types?

2. Data Loading and Quality Checks

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Campaign-Data.csv to Campaign-Data.csv


In [3]:
data = pd.read_csv("Campaign-Data.csv")
data.columns

Index(['Client ID', 'Client Type', 'Number of Customers', 'Montly Target',
       'Zip Code', 'Calendardate', 'Amount Collected', 'Unit Sold',
       'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5', 'Number of Competition'],
      dtype='object')

In [4]:
data.head()

,Client ID,Client Type,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition
0,ID-987275,Medium Facility,2800,125,1003,16-01-2014,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
1,ID-987275,Medium Facility,2800,125,1003,16-02-2014,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,Low
2,ID-987275,Medium Facility,2800,125,1003,18-03-2014,10228384,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
3,ID-987275,Medium Facility,2800,125,1003,18-04-2014,17047304,123,0.0,0.0,0.0,0.0,3547500.0,1290000.0,0.0,0.0,Low
4,ID-987275,Medium Facility,2800,125,1003,19-05-2014,23866224,171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low


In [5]:
data.dtypes

,0
Client ID,object
Client Type,object
Number of Customers,int64
Montly Target,int64
Zip Code,int64
Calendardate,object
Amount Collected,int64
Unit Sold,int64
Campaign (Email),float64
Campaign (Flyer),float64


In [6]:
# Rename "Montly Target" column to "Monthly Target"

data.rename(columns={'Montly Target':'Monthly Target'}, inplace=True)
data.head(2)

,Client ID,Client Type,Number of Customers,Monthly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition
0,ID-987275,Medium Facility,2800,125,1003,16-01-2014,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
1,ID-987275,Medium Facility,2800,125,1003,16-02-2014,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,Low


In [7]:
# Clearing any trailing white space
def remove_space(x):
    if isinstance(x, str):
        return x.strip()
    else:
        return x

data = data.map(remove_space)

3. Feature Additions and Engineering

In [8]:
# Converting the 'Calenderdate' column to a datetime format and creating new columns 'Calendar_Month' and 'Calendar_Year'

data['Calendardate'] = pd.to_datetime(data['Calendardate'])
data['Calendar_Month'] = data['Calendardate'].dt.month
data['Calendar_Year'] = data['Calendardate'].dt.year

<ipython-input-8-ce9c560d4321>:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Calendardate'] = pd.to_datetime(data['Calendardate'])


4. Exploratory Data Analysis and Statistical Analysis

We can have a detailed exploration that can be added to this section, but since we only need to answer three questions:

4.1 Exploring and Understanding basics data
  1. Distribution of Data across different accounts
  2. Difference of Sales in Account Types (Using Categorical Mean)

4.2 Statistical Analysis - Answering the Questions
  1. Impact of Marketing Strategy on Sales (Using Correlation, Regression and Decision Tree)
  2. Impact of Competition on Sales
  3. How different types of clients can have different strategies (Broken down question 1 and question 2 based on Account Type)

4.2 Impact of Marketing Strategy on Sales

  Understanding of distributions

In [9]:
# client type distributions
data['Client Type'].value_counts(normalize=True)

,proportion
Client Type,
Large Facility,0.459677
Small Facility,0.282258
Medium Facility,0.169355
Private Facility,0.088710


In [10]:
pd.crosstab(data['Number of Competition'], data['Client Type'], margins=True, normalize='columns')

Client Type,Large Facility,Medium Facility,Private Facility,Small Facility,All
Number of Competition,,,,,
High,0.166667,0.166667,0.166667,0.166667,0.166667
Low,0.833333,0.833333,0.833333,0.833333,0.833333


In [11]:
data.groupby('Client Type')[['Amount Collected', 'Unit Sold', 'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5',]].mean()

,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5
Client Type,,,,,,,,,,
Large Facility,1.999880e+07,143.098684,142273.609649,8.192056e+05,45595.436623,133667.763158,2.034013e+06,2.017039e+06,119287.280702,16266.447368
Medium Facility,4.075997e+07,290.583333,437217.097817,1.552603e+06,49176.847619,398645.833333,4.822783e+06,4.698646e+06,85104.166667,33273.809524
Private Facility,5.030246e+06,35.784091,5183.715152,2.272919e+05,5522.470455,1221.590909,6.376705e+05,4.434375e+05,3664.772727,12215.909091
Small Facility,1.637759e+06,11.689286,11975.986310,9.120875e+04,0.000000,8062.500000,7.617143e+05,3.727946e+05,4223.214286,1535.714286
